# 네이버 영화평 감성 분석

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

- 데이터 가져오기 : '네이버 영화 리뷰 데이터'로 검색

In [2]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [3]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


-데이터 전처리

-트레인 데이터 셋

In [4]:
#중복여부 확인
train_df.document.nunique()

146182

In [5]:
#중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [6]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [7]:
#null 데이터 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [8]:
# 부정, 긍정 데이터 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋

In [9]:
#중복여부확인
test_df.document.nunique()

49157

In [10]:
# Null 데이터가 있는지 확인
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [11]:
#null 데이터 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49997, 3)

2.텍스트 전처리

- train dataset

In [12]:
# 한글과 공백 의외의 문자는 제거
train_df.document = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [13]:
# 데이터가 다 지워져서 ''가 된 데이터 --> np.nan으로 변경한 후 제거
train_df.document.replace('',np.nan, inplace=True)
train_df.isna().sum()


id            0
document    391
label         0
dtype: int64

In [14]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

In [15]:
test_df.document = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df.document.replace('',np.nan, inplace=True)
test_df = test_df.dropna(how='any')
test_df.shape

(49726, 3)

- 전처리가 끝난 데이터는 저장

In [16]:
train_df.to_csv('data/never_movie_tran.tsv', sep='\t', index=False)
test_df.to_csv('data/never_movie_tran.tsv', sep='\t', index=False)

3. 한글처리

In [17]:
from konlpy.tag import Okt
okt = Okt()

In [18]:
text = '흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나'
okt.morphs(text)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍지', '않구나']

In [19]:
okt.morphs(text, stem=True)

['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']

In [20]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [21]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()

In [22]:
from tqdm.notebook import tqdm
X_train = []
for sentence in tqdm(train_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_list = [word for word in morphs if word not in stopwords]
    tmp_str = ' '.join(tmp_list)
    X_train.append(tmp_str)

  0%|          | 0/145791 [00:00<?, ?it/s]

In [ ]:
X_test = []
for sentence in tqdm(test_df.document):
    morphs = okt.morphs(sentence, stem=True)
    tmp_str = ' '.join([word for word in morphs if word not in stopwords])
    X_test.append(tmp_str)

  0%|          | 0/49726 [00:00<?, ?it/s]

In [ ]:
y_train = train_df.label.values
y_test = test_df.label.values

#### 4. Feature 변환

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((145791, 42115), (49726, 42115))

In [ ]:
#### 5. 모델 학습/평가
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_cv, y_train)
lr.score(X_test_cv, y_test)

AttributeError: 'str' object has no attribute 'decode'

6. 실제 테스트

In [ ]:
review1 = '두시간 반이 왜 이렇게 짧냐'
review2 = '저 정말 스파이더맨 팬입니다. 정말 역겨움을 참느라 힘들었습니다.'

In [ ]:
# 전처리
import re
review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review1)
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review2)

In [ ]:
#불용어 처리
morphs = okt.morphs(review1, stem=True)
review1 = ' '.join(word for word in morphs if word not in stopwords)
review1

'두 시간 반 왜 이렇게 짧다'

In [ ]:
morphs = okt.morphs(review2, stem=True)
review2 = ' '.join(word for word in morphs if word not in stopwords)
review2

'두 시간 반 왜 이렇게 짧다'

In [ ]:
# Feature 변환
review1_cv = cvect.transform([review1])
lr.predict(review1_cv)

In [ ]:
review2_cv = cvect.transform([review2])
lr.predict(review2_cv)